Exercise 4 - Polynomial Regression
===

Sometimes our data doesn't have a linear relationship, but we still want to predict an outcome.

Suppose we want to predict how satisfied people might be with a piece of fruit. We would expect satisfaction would be low if the fruit is under-ripe or over-ripe, and satisfaction would be high if fruit is ripe, i.e. between under-ripe and over-ripe.

This is not something linear regression will help us with, however polynomial regression can help us make predictions for more complex non-linear relationships such as these.

Step 1
---

In this exercise, we will look at a dataset analysing internet traffic over the course of the day. Observations were made every hour, on the hour, over the course of several days. Suppose we want to predict the level of internet traffic we might see at any time during the day, how might we do this?

Let's start by loading the required libraries for this session, and loading our data to have a look at it.

** In the cell below replace: **

** 1. `<dataPreviewStr>` with `str` **

** 2. `<dataPreviewHead>` with `head` **

** then __run the code__. **

In [ ]:
# Load required libraries
suppressMessages(library("tidyverse"))

traf_by_hr <- read.csv("Data/traffic_by_hour.csv", check.names = FALSE)

###
# REPLACE <dataPreviewStr> WITH str AND <dataPreviewHead> WITH head
###
<dataPreviewStr>(traf_by_hr)
<dataPreviewHead>(traf_by_hr)
###

By observing the structure of `traf_by_hr`, we can observe that we have the following variables: 

* `Hour` 00 - 23, spread across the columns;
* `Observation` 1 - 6, with each observation representing one row;
* `Traffic` (in units Gbps), representing the values of each observation made every hour. 

Step 2
---

Next, we need to reshape the data using the package `tidyr` so we can plot our data using `ggplot2`. The `ggplot2` functions expect our data input to be in "long" format, i.e. a column for each feature name, and a row for each observation. Currently, our data is stored in "wide" format, with one feature, i.e. the hour, spread across multiple columns. We need to reshape the `traf_by_hr` data so that our variables, `Hour`, `Observation`, and `Traffic` are the column names.

> "Long" and "wide" format are visual metaphors that describe two ways of presenting the exact same information. Hopefully you will understand the metaphor once you see "long" and "wide" data for yourself, as per below!

#### Run the following code block to reshape `traf_by_hr` ("wide" format) to `traf_by_hr_long` ("long" format). You do not need to edit the code block.

In [ ]:
# Run this!

# DO NOT EDIT

# Reshape data to long format using tidyr's gather function
traf_by_hr_tall <- traf_by_hr %>%
gather(key = "Hour", value = "Traffic") %>%
mutate(Observation = as.factor(rep(1:6, 24)))  %>% 
mutate(Hour = as.numeric(Hour))

# Check structure of reshaped data
str(traf_by_hr_tall)
head(traf_by_hr_tall, n = 10)

Compare the structural difference between `traf_by_hr_long` and `traf_by_hr` for yourself, particularly the dimensions. You should observe the following:

* `traf_by_hr` ("wide" data): 6 observations (rows) x 24 variables (columns);
* `traf_by_hr_long` ("long" data): 144 observations (rows) x 3 variables (columns).

Now with our data in "long" format, we can use `ggplot2` functions for plotting.

In the `ggplot` function call within the code block below:

** In the cell below replace: **  
** 1. `<xData>` with `Hour` **  
** 2. `<yData>` with `Traffic` **  
** then __run the code__. **

In [ ]:
# Visualise the data using ggplot2
traf_by_hr_tall  %>% 

###
# REPLACE THE <xData> TO Hour AND <yData> TO Traffic
###
ggplot(aes(x = <xData>, y = <yData>, colour = Observation)) +
###
geom_line() +
ggtitle("Internet traffic for each hour of the day") +
xlab("Hour of the day (24 hour time)") +
ylab("Internet traffic (Gbps)") +
theme(plot.title = element_text(hjust = 0.5))

This plot looks a bit busy due to overplotting. We should summarize the data to help us visualize trends.

Step 3
---

Let's see if we can visualize a clearer pattern by taking the __average values__ for each hour.

In the code block below, find the `geom_point` function call, then:

** In the cell below replace:** 

**1. `<xData>` with `Hour`**  

**2. `<yData>` with `Traffic_mean`**  

**then __run the code__. **

In [ ]:
# Find mean values for each hour using `dplyr` function `mutate`
traf_by_hr_tall <- traf_by_hr_tall  %>% 
group_by(Hour)  %>% 
mutate(Traffic_mean = mean(Traffic))  %>% 
ungroup()  %>% 
as.data.frame()

# Check structure of data
str(traf_by_hr_tall)
head(traf_by_hr_tall, n = 10)

# Create plot
traf_by_hr_tall  %>% 
ggplot() +

# Plot the average of the 6 observations as points

###
# REPLACE <xData> TO Hour and <yData> to Traffic_mean
###
geom_point(aes(x = <xData>, y = <yData>)) +
###
# Plot each observation 1 - 6 as a line
geom_line(aes(x = Hour, y = Traffic, colour = Observation), alpha = 0.5) +
ylab("Average internet traffic (Gbps)") +
theme(plot.title = element_text(hjust = 0.5))

The plot above shows the average value for each hour as points (black), together with observations 1 - 6 as lines.

We can also plot our data using a graph type that summarizes the data for us, such as a box and whisker plot.

In the code below, within the `ggplot` function call, change the x and y variables to the features we want to observe.

** In the cell below replace:**  
**1. `<xData>` with `Hour`**  
**2. `<yData>` with `Traffic`**  
**then __run the code__. **

In [ ]:
# Create box and whisker plot
traf_by_hr_tall  %>% 
###
# REPLACE <xData> TO Hour and <yData> to Traffic
###
ggplot(aes(x = <xData>, y = <yData>, group = Hour)) +
###
geom_boxplot() +
ggtitle("Internet traffic for each hour of the day") +
xlab("Hour of the day (24 hour time)") +
ylab("Internet traffic (Gbps)") +
# Align title to centre
theme(plot.title = element_text(hjust = 0.5))

This summarization of the data could help us make a prediction if we wanted to know the expected traffic exactly on the hour.

But, we'll need to be a bit clever if we want to make a good prediction of times in between.

Step 4
---

Let's use the midpoints in between the hours to help us analyse the relationship between the time of day and the amount of internet traffic.

The `lm` (linear model) function together with the `poly` (polynomial) function allow us to do just this. We need to specify a feature $x$ (time of day), our outcome $y$ (the amount of internet traffic), and the $degree$ of the polynomial (how curvy the line is).

> You can use the `lm` function directly, but for this exercise we will use `lm` indirectly through `ggplot2`.

First we will test polynomial functions with degrees 1, 2, 3 and 4. Note the first degree polynomial (degree = 1) has already been completed for you; first degree polynomials are linear, so we will include it for comparison.

** In the cell below replace: **  
** 1. `<addDegree2>` with `2`**  
** 2. `<addDegree3>` with `3`**  
** 3. `<addDegree4>` with `4`**  
** then __run the code__.**

In [ ]:
traf_by_hr_tall  %>% 
ggplot(aes(x = Hour, y = Traffic_mean)) +
geom_point(alpha = 0.5) +
stat_smooth(method = "lm", formula = y ~ poly(x, degree = 1), colour = "black", linetype = "dashed", se = FALSE) +

###
# REPLACE <addDegree2> WITH 2, <addDegree3> WITH 3, AND <addDegree4> WITH 4
###
stat_smooth(method = "lm", formula = y ~ poly(x, degree = <addDegree2>), colour = "#F8766D", se = FALSE) + # red
stat_smooth(method = "lm", formula = y ~ poly(x, degree = <addDegree3>), colour = "#00BFC4", se = FALSE) + # blue
stat_smooth(method = "lm", formula = y ~ poly(x, degree = <addDegree4>), colour = "#7CAE00", se = FALSE) + # green

xlab("Hour of the day (24 hour time)") +
ylab("Internet traffic (Gbps)") +
ggtitle("Testing fit of polynomial functions (degrees 1 - 4) to internet traffic data") +
theme(plot.title = element_text(hjust = 0.5))

None of these polynomial functions do a great job of generalising the data. Let's try a few more.

In the code below, test polynomial functions of degree 5, 6 and 7.

** In the cell below replace:**  
** 1. `<addDegree5>` with `5`**  
** 2. `<addDegree6>` with `6`**  
** 3. `<addDegree7>` with `7`**  
** then __run the code__.**

In [ ]:
traf_by_hr_tall  %>% 
ggplot(aes(x = Hour, y = Traffic_mean)) +
geom_point(alpha = 0.5) +
# Polynomials of degree 1 are linear, keep this for comparison
stat_smooth(method = "lm", formula = y ~ poly(x, degree = 1), colour = "black", linetype = "dashed", se = FALSE) +
# Change degree = ? to degree = 5

###
# REPLACE <addDegree5> WITH 5, <addDegree6> WITH 6, AND <addDegree7> WITH 7
###
stat_smooth(method = "lm", formula = y ~ poly(x, degree = <addDegree5>), colour = "#F8766D", se = FALSE) + # red
stat_smooth(method = "lm", formula = y ~ poly(x, degree = <addDegree6>), colour = "#00BFC4", se = FALSE) + # blue
stat_smooth(method = "lm", formula = y ~ poly(x, degree = <addDegree7>), colour = "#7CAE00", se = FALSE) + # green
###

xlab("Hour of the day (24 hour time)") +
ylab("Internet traffic (Gbps)") +
ggtitle("Testing fit of polynomial functions (degrees 5 - 7) to internet traffic data") +
theme(plot.title = element_text(hjust = 0.5))

It looks like the 6th and 7th degree polynomials have an identical curve, so either of these polynomials will be a good model to use.

We could use an even higher degree polynomial to fit the model to our data even more tightly, but we don't want to overfit the curve, since we just want a generalization of the relationship between time of day and internet traffic.

Let's see how our 6th degree polynomial alone compares to the real data.

#### Run the code below.

In [ ]:
# Run this!

# DO NOT EDIT

traf_by_hr_tall  %>% 
ggplot(aes(x = Hour, y = Traffic, colour = Observation)) +
geom_line(alpha = 0.5) +
stat_smooth(method = "lm", formula = y ~ poly(x, degree = 6), colour = "black", se = FALSE) +
xlab("Hour of the day (24 hour time)") +
ylab("Internet traffic (Gbps)") +
ggtitle("Testing fit of 6th degree polynomial to internet traffic data") +
theme(plot.title = element_text(hjust = 0.5))

Looking good!

Step 5
---

Now let's try using this polynomial regression model to make a prediction for how much internet traffic there will be at a certain time of day. Let's choose the time 12:30pm.

** Replace the `<addTime>` with `12.5`, which represents the time 12.30 pm, and run the code. **

In [ ]:
###
# REPLACE THE <addTime> WITH 12.5 (THIS REPRESENTS 12:30PM)
###
t <- data.frame(Hour = <addTime>)
###

# Save our 6th degree polynomial model
lm_poly_6th <- lm(formula = Traffic ~ poly(Hour, degree = 6), data = traf_by_hr_tall)

# Use predict function, and round the result to 2 decimal places
# Input to predict function must be data frame, with column name set as x value
t_pred <- round(predict(lm_poly_6th, t), 2)

print(paste("Based on our polynomial regression model, at time t = 12.5",
      "the expected internet traffic is", t_pred , "Gbps."))

traf_by_hr_tall %>% 
ggplot(aes(x = Hour, y = Traffic)) +
stat_smooth(method = "lm", formula = y ~ poly(x, degree = 6), colour = "black", se = FALSE) +
# Show predicted value as a point in red
geom_point(x = 12.5, y = t_pred, size = 3) +
# Add horizontal line
geom_hline(yintercept = t_pred, linetype = "dashed", colour = "red") +
geom_vline(xintercept = 12.5, linetype = "dashed", colour = "red") +
ggtitle("Prediction of expected traffic at t = 12.5") +
xlab("Hour of the day (24 hour time)") +
ylab("Internet traffic (Gbps)") +
theme(plot.title = element_text(hjust = 0.5))

Conclusion
---

There we have it! You have made a polynomial regression model and used it for analysis! This models gives us a prediction for the level of internet traffic we should expect to see at any given time of the day.

You can go back to the course and either click __'Next Step'__ to start an optional step with tips on how to better work with AI models, or you can go to the next module where instead of predicting numbers we predict categories.